In [1]:
from __future__ import annotations
import math
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Tuple, Iterable, Optional
from scipy.stats import norm

# Quick sabr calibration test
### basically, generates fake market vols, runs a simple random+refine search 
### to fit (alpha, rho, nu) for each expiry (fixed beta), 
### then checks the atm vol + a black price just to see if it's sane
a toy example to get the taste

In [2]:
def _rmse(y: np.ndarray, yhat: np.ndarray, w: Optional[np.ndarray]=None) -> float:
    if w is None:
        return float(np.sqrt(np.mean((y - yhat)**2)))
    return float(np.sqrt(np.mean((w * (y - yhat))**2)))

@dataclass
class SABRSmileInput:
    T: float
    F: float
    strikes: np.ndarray
    vols: np.ndarray      
    df: float = 1

In [3]:
def black_call(F, K, T, sigma_bs, df=1):
    if T <= 0 or sigma_bs <= 0: # for just in case
        return max(df * (F - K), 0)
    v = sigma_bs * np.sqrt(T)
    d1 = (np.log(F / K) + 0.5 * v * v) / v
    d2 = d1 - v
    return df * (F * norm.cdf(d1) - K * norm.cdf(d2))

def bachelier_call(F, K, T, sigma_n, df=1):
    if T <= 0 or sigma_n <= 0: # for just in case
        return max(df * (F - K), 0)
    s = sigma_n * np.sqrt(T)
    d = (F - K) / s
    return df * ((F - K) * norm.cdf(d) + s * norm.pdf(d))

In [4]:
def hagan_logn_sabr_vol(F: float, K: float, T: float,
                        alpha: float, beta: float, rho: float, nu: float) -> float:
    """
    Hagan et al. (2002) lognormal SABR implied volatility
    """
    if T <= 0: # debug later
        return 0
    F = max(F, 1e-16)
    K = max(K, 1e-16)
    one_minus_beta = 1 - beta

    if abs(F - K) < 1e-12:
        F1mb = F**(one_minus_beta)
        term1 = (one_minus_beta**2 / 24) * (alpha**2) / (F**(2 * one_minus_beta))
        term2 = (rho * beta * nu * alpha) / (4 * F1mb)
        term3 = ((2 - 3 * rho**2) * (nu**2) / 24)
        sigma_atm = alpha / F1mb * (1 + (term1 + term2 + term3) * T)
        return max(sigma_atm, 0)

    logFK = math.log(F / K)
    FK = F * K
    D = (FK)**(0.5 * one_minus_beta) * (
        1
        + (one_minus_beta**2 / 24) * (logFK**2)
        + (one_minus_beta**4 / 1920) * (logFK**4)
    )
    z = (nu / alpha) * (FK)**(0.5 * one_minus_beta) * logFK
    sqrt_term = math.sqrt(1 - 2 * rho * z + z * z)
    xz = math.log((sqrt_term + z - rho) / (1 - rho))

    term1 = (one_minus_beta**2 / 24) * (alpha**2) / (FK**(one_minus_beta))
    term2 = (rho * beta * nu * alpha) / (4 * (FK)**(0.5 * one_minus_beta))
    term3 = ((2 - 3 * rho**2) * (nu**2) / 24)

    sigma = (alpha / D) * (z / xz) * (1 + (term1 + term2 + term3) * T)
    return max(sigma, 0)

In [5]:
def hagan_normal_sabr_vol(Fs: float, Ks: float, T: float,
                          alpha: float, rho: float, nu: float) -> float:
    """
    Normal (Bachelier) implied vol for SABR with beta=0 or shifted SABR
    """
    if T <= 0: # for just in case again
        return 0
    z = (nu / alpha) * (Fs - Ks)
    sqrt_term = math.sqrt(1 - 2*rho*z + z*z)
    xz = math.log((sqrt_term + z - rho) / (1 - rho)) if abs(z) > 1e-12 else 1
    return max(alpha * (z / xz) * (1 + ((2 - 3*rho*rho) * nu*nu / 24) * T), 0)  

In [6]:
def calibrate_sabr_smile(
    smile: SABRSmileInput,
    beta: float,
    normal: bool = False,
    shift: float = 0,
    n_random: int = 1500,
    n_refine: int = 300,
    seed: int = 777
) -> Tuple[Dict[str,float], float]:
    """
    Calibrates (alpha, rho, nu) for a fixed beta to market vols at a single maturity
    """
    rng = np.random.default_rng(seed)
    F0 = smile.F
    Ks = np.asarray(smile.strikes)
    sig_mkt = np.asarray(smile.vols)
    T = smile.T

    def model_vols(alpha, rho, nu):
        if normal:
            Fs = F0 + shift
            Ks_s = Ks + shift
            return np.array([hagan_normal_sabr_vol(Fs, k, T, alpha, rho, nu) for k in Ks_s])
        else:
            return np.array([hagan_logn_sabr_vol(F0, k, T, alpha, beta, rho, nu) for k in Ks])

    def obj(x):
        a, r, n = x
        if not (1e-8 <= a <= 5 and -0.999 <= r <= 0.999 and 1e-8 <= n <= 5):
            return 1e6
        sig_model = model_vols(a, r, n)
        # I call it "heuristic" here
        w = 1 / np.maximum(sig_mkt, 1e-6)
        return _rmse(sig_mkt, sig_model, w=w)

    box_a = (0.02, 0.8)
    box_r = (-0.9, 0.9)
    box_n = (0.05, 1.5)

    A = rng.uniform(*box_a, size=n_random)
    R = rng.uniform(*box_r, size=n_random)
    N = rng.uniform(*box_n, size=n_random)
    C = np.stack([A, R, N], axis=1)

    vals = np.array([obj(c) for c in C])
    x = C[int(np.argmin(vals))].copy()
    best = float(np.min(vals))

    step = np.array([0.15, 0.25, 0.40])
    for _ in range(n_refine):
        improved = False
        for d in range(3):
            for sgn in (+1, -1):
                trial = x.copy()
                trial[d] += sgn * step[d]
                trial = np.array([
                    float(np.clip(trial[0], *box_a)),
                    float(np.clip(trial[1], *box_r)),
                    float(np.clip(trial[2], *box_n)),
                ])
                val = obj(trial)
                if val + 1e-12 < best:
                    x, best, improved = trial, val, True
        if not improved:
            step *= 0.7
            if np.max(step) < 1e-4:
                break

    alpha, rho, nu = map(float, x)
    return {"alpha": alpha, "rho": rho, "nu": nu, "beta": float(beta)}, best

In [7]:
def build_fitted_surface(
    smiles: Iterable[SABRSmileInput],
    beta: float,
    normal: bool = False,
    shift: float = 0,
    **calib_kwargs
) -> Dict[float, Dict[str,float]]:
    """
    Calibrate each maturity independently (fixed beta)
    """
    result = {}
    for sm in smiles:
        pars, rmse = calibrate_sabr_smile(sm, beta=beta, normal=normal, shift=shift, **calib_kwargs)
        pars["rmse"] = rmse
        pars["F"] = sm.F
        result[sm.T] = pars
    return result

In [8]:
# toy example
beta = 0.8
maturities = [1, 2, 3, 4, 5]  # in years
forwards   = {
    1: 0.025,
    2: 0.0275,
    3: 0.0285,
    4: 0.0294,
    5: 0.03
}
m_grid = np.array([-0.1, -0.05, -0.02, -0.01, 0, 0.01, 0.02, 0.05, 0.1])
def strikes_from_m(F, m_arr): 
    return F*np.exp(m_arr)

In [9]:
true_params = { # as if, in real life we take it fromdata vendor
    1: dict(alpha=0.22, rho=-0.35, nu=0.55),
    2: dict(alpha=0.2, rho=-0.30, nu=0.5),
    3: dict(alpha=0.19, rho=-0.27, nu=0.49),
    4: dict(alpha=0.16, rho=-0.22, nu=0.44),
    5: dict(alpha=0.13, rho=-0.2, nu=0.4),
}

smiles = []
for T in maturities:
    F0 = forwards[T]
    Ks = strikes_from_m(F0, m_grid)
    tp = true_params[T]
    vols = np.array([hagan_logn_sabr_vol(F0, K, T, tp["alpha"], beta, tp["rho"], tp["nu"]) for K in Ks])
    vols *= (1 + 0.01*np.random.randn(*vols.shape))
    smiles.append(SABRSmileInput(T=T, F=F0, strikes=Ks, vols=vols))

fitted = build_fitted_surface(smiles, beta=beta, normal=False, n_random=1500, n_refine=300, seed=111)

for T in maturities:
    p = fitted[T]
    t = true_params[T]
    print(f"T={T:>4.0f}y | alpha={p['alpha']:.6f} (true {t['alpha']:.3f})  "
            f"rho={p['rho']:.6f} (true {t['rho']:.2f})  "
            f"nu={p['nu']:.6f} (true {t['nu']:.2f})   RMSE={p['rmse']:.6e}")

T=   1y | alpha=0.200311 (true 0.220)  rho=-0.146090 (true -0.35)  nu=1.157672 (true 0.55)   RMSE=9.826505e-03
T=   2y | alpha=0.197965 (true 0.200)  rho=-0.257640 (true -0.30)  nu=0.520845 (true 0.50)   RMSE=8.237338e-03
T=   3y | alpha=0.190799 (true 0.190)  rho=-0.374820 (true -0.27)  nu=0.591531 (true 0.49)   RMSE=8.584423e-03
T=   4y | alpha=0.168687 (true 0.160)  rho=-0.616619 (true -0.22)  nu=0.099723 (true 0.44)   RMSE=1.212210e-02
T=   5y | alpha=0.130248 (true 0.130)  rho=-0.250107 (true -0.20)  nu=0.426638 (true 0.40)   RMSE=6.283179e-03


In [10]:
T_ex = 3
F_ex = forwards[T_ex]
K_ex = F_ex
df_ex = 1
par = fitted[T_ex]
vol_atm = hagan_logn_sabr_vol(F_ex, K_ex, T_ex, par["alpha"], beta, par["rho"], par["nu"])
px = black_call(F_ex, K_ex, T_ex, vol_atm, df=df_ex)
print(f"Example (T={T_ex}y, K=F):   imp vol={vol_atm:.6f},   Black call PV={px:.8f}")

Example (T=3y, K=F):   imp vol=0.395725,   Black call PV=0.00764319
